In [1]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import ast

# Carica CSV
fd = pd.read_csv('TrackFeatures4.csv')

In [2]:
def string_to_dict(fd, key):
    """Converte una stringa in un dizionario Python"""
    print(f"\nConverto le stringhe di '{key}' contenenti mean e std in dizionari Python")
    parsed = fd[key].apply(ast.literal_eval)
    mean = parsed.apply(lambda x: x['mean'])
    std = parsed.apply(lambda x: x['std'])
    return mean, std

# Appiattisci una matrice 2D in un vettore 1D
def flatten_feature_matrix(matrix):
    return np.array(matrix).flatten()
i=0
# Funzione per il preprocessing di una singola riga
def preprocess_row(row):
    global i
    print(f"\nPreprocessing riga {i}...")

    i+=1
    try:
        mfccs = ast.literal_eval(row['mfccs'])
        chroma = ast.literal_eval(row['chroma'])
        spec_contrast = ast.literal_eval(row['spec_contrast'])
        zcr = ast.literal_eval(row['zcr'])
        beats = ast.literal_eval(row['beats'])
        tempo_feature = float(row['tempo'].replace("[", "").replace("]", ""))

        mfccs_vec = np.concatenate([
            flatten_feature_matrix(mfccs['mean']),
            flatten_feature_matrix(mfccs['std'])
        ])

        chroma_vec = np.concatenate([
            flatten_feature_matrix(chroma['mean']),
            flatten_feature_matrix(chroma['std'])
        ])

        spec_vec = np.concatenate([
            flatten_feature_matrix(spec_contrast['mean']),
            flatten_feature_matrix(spec_contrast['std'])
        ])

        zcr_vec = np.concatenate([
            flatten_feature_matrix(zcr['mean']),
            flatten_feature_matrix(zcr['std'])
        ])

        beats_vec = np.array([beats['count'], beats['interval_mean'], beats['interval_std']])

        # Unisci tutte le feature in un vettore
        return np.concatenate([mfccs_vec, chroma_vec, spec_vec, zcr_vec, [tempo_feature], beats_vec])
    except Exception as e:
        print(row)
        print(f"Errore nel preprocessing della riga {i}: {e}")
        return np.zeros(5944)  # Restituisci un vettore di zeri della lunghezza corretta in caso di errore


In [89]:
# Funzione per convertire la stringa in dizionario
# Applichiamo il preprocessing a tutte le righe
print("Preprocessing in corso...")
feature_matrix = fd.apply(preprocess_row, axis=1)
X = np.stack(feature_matrix.to_numpy())  # Convertiamo in matrice numpy

# Label (target)
y = fd[fd.columns[1]]  # Etichetta dei generi musicali
print(f"\n\nMatrix X: {X.shape}")



Preprocessing in corso...

Preprocessing riga 0...

Preprocessing riga 1...

Preprocessing riga 2...

Preprocessing riga 3...

Preprocessing riga 4...

Preprocessing riga 5...

Preprocessing riga 6...

Preprocessing riga 7...

Preprocessing riga 8...

Preprocessing riga 9...

Preprocessing riga 10...

Preprocessing riga 11...

Preprocessing riga 12...

Preprocessing riga 13...

Preprocessing riga 14...

Preprocessing riga 15...

Preprocessing riga 16...

Preprocessing riga 17...

Preprocessing riga 18...

Preprocessing riga 19...

Preprocessing riga 20...

Preprocessing riga 21...

Preprocessing riga 22...

Preprocessing riga 23...

Preprocessing riga 24...

Preprocessing riga 25...

Preprocessing riga 26...

Preprocessing riga 27...

Preprocessing riga 28...

Preprocessing riga 29...

Preprocessing riga 30...

Preprocessing riga 31...

Preprocessing riga 32...

Preprocessing riga 33...

Preprocessing riga 34...

Preprocessing riga 35...

Preprocessing riga 36...

Preprocessing riga 37

In [90]:
print(f"Label y: {set(y)}")

Label y: {'Reggae', 'Jazz', 'Funk', 'Soul', 'House', 'Dubstep', 'Electronic', 'Blues', 'Disco', 'Pop', 'Classical', 'Hip-Hop', 'Drum and Bass', 'R&B', 'Techno', 'Country', 'Folk', 'Punk', 'Metal', 'Trance', 'Rock'}


In [91]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import joblib
# === 1. Normalizzazione delle feature (StandardScaler) ===
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
joblib.dump(scaler, "scaler_sicuro.pkl")


['scaler_sicuro.pkl']

In [92]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)  # y → array di interi
joblib.dump(scaler, "labelEncoder_sicuro.pkl")


['labelEncoder_sicuro.pkl']

In [93]:
# # === 1. Prima split: 95% train_val, 5% test ===
# X_train_val, X_test, y_train_val, y_test = train_test_split(
#     X_scaled, y_encoded, test_size=0.05, random_state=42, stratify=y_encoded
# )

# # === 2. Seconda split: da train_val → 75% train, 20% val ===
# # Calcolo proporzione della val set su train_val (20 / 95 ≈ 0.2105)
# val_size = 0.20 / 0.95

# X_train, X_val, y_train, y_val = train_test_split(
#     X_train_val, y_train_val, test_size=val_size, random_state=42, stratify=y_train_val
# )

X_training, X_val, y_training, y_val = train_test_split(
    X_scaled, y_encoded, test_size=0.2, random_state=123, stratify=y_encoded
)

In [94]:
X_train_tensor = torch.tensor(X_training, dtype=torch.float32)
X_val_tensor   = torch.tensor(X_val, dtype=torch.float32)
#X_test_tensor  = torch.tensor(X_test, dtype=torch.float32)

y_train_tensor = torch.tensor(y_training, dtype=torch.long)
y_val_tensor   = torch.tensor(y_val, dtype=torch.long)
#y_test_tensor  = torch.tensor(y_test, dtype=torch.long)


In [95]:
import torch.nn as nn

# Numero di feature in input
input_size = X_train_tensor.shape[1]
print(f"\nNumero di feature in input: {input_size}")
num_classes = len(np.unique(y_encoded))
print(f"Numero di classi: {num_classes}")

# Modello semplice: input → 128 → ReLU → 64 → ReLU → output
class MusicGenreClassifier(nn.Module):
    def __init__(self):
        super(MusicGenreClassifier, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, 1000),
            nn.ReLU(),
            nn.Linear(1000, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            #nn.ReLU(),
            #nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        return self.net(x)

model = MusicGenreClassifier()



Numero di feature in input: 5944
Numero di classi: 21


In [96]:
epochs = 50
lr = 0.001
print(f"\nLearning rate: {lr}")
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    # Valutazione su validation set
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val_tensor)
        val_loss = criterion(val_outputs, y_val_tensor)
        val_preds = torch.argmax(val_outputs, dim=1)
        val_acc = (val_preds == y_val_tensor).float().mean()

    print(f"Epoch {epoch+1}/{epochs} | Train Loss: {loss.item():.4f} | Val Loss: {val_loss.item():.4f} | Val Acc: {val_acc:.4f}")



Learning rate: 0.001
Epoch 1/50 | Train Loss: 3.0361 | Val Loss: 2.8088 | Val Acc: 0.1602
Epoch 2/50 | Train Loss: 2.7912 | Val Loss: 2.5820 | Val Acc: 0.1905
Epoch 3/50 | Train Loss: 2.5610 | Val Loss: 2.4081 | Val Acc: 0.2537
Epoch 4/50 | Train Loss: 2.3391 | Val Loss: 2.2933 | Val Acc: 0.3022
Epoch 5/50 | Train Loss: 2.1848 | Val Loss: 2.2347 | Val Acc: 0.3221
Epoch 6/50 | Train Loss: 2.0756 | Val Loss: 2.1612 | Val Acc: 0.3316
Epoch 7/50 | Train Loss: 1.9570 | Val Loss: 2.0595 | Val Acc: 0.3654
Epoch 8/50 | Train Loss: 1.8165 | Val Loss: 2.0093 | Val Acc: 0.3853
Epoch 9/50 | Train Loss: 1.7235 | Val Loss: 1.9179 | Val Acc: 0.4087
Epoch 10/50 | Train Loss: 1.5902 | Val Loss: 1.8669 | Val Acc: 0.4173
Epoch 11/50 | Train Loss: 1.4899 | Val Loss: 1.8148 | Val Acc: 0.4320
Epoch 12/50 | Train Loss: 1.3958 | Val Loss: 1.7640 | Val Acc: 0.4494
Epoch 13/50 | Train Loss: 1.2904 | Val Loss: 1.7586 | Val Acc: 0.4528
Epoch 14/50 | Train Loss: 1.1975 | Val Loss: 1.7364 | Val Acc: 0.4649
Epoch 1

In [97]:
model.eval()
with torch.no_grad():
    test_outputs = model(X_val_tensor)
    test_preds = torch.argmax(test_outputs, dim=1)
    test_acc = (test_preds == y_val_tensor).float().mean()

print(f"✅ Test Accuracy: {test_acc:.4f}")


✅ Test Accuracy: 0.5091


In [98]:
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_preds = torch.argmax(test_outputs, dim=1)  # Indici delle classi predette

# Converti in etichette
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(fd[fd.columns[1]])  # Dove `fd.columns[1]` è la colonna dei generi

predicted_labels = label_encoder.inverse_transform(test_preds.numpy())

print(predicted_labels[:10])  # Stampa i primi 10 generi predetti


NameError: name 'X_test_tensor' is not defined

In [ ]:
print(X_test)

[[ 1.10836521  1.02699526  0.97695506 ...  1.54786814 -0.91114812
  -0.45373743]
 [-0.60175117  0.03544164  0.09765353 ... -0.39100477  0.1975857
   0.84108338]
 [ 0.82161626  0.70127059  0.76344635 ...  0.57843168 -0.5304248
  -0.71449214]
 ...
 [ 1.30657146  1.21683709  1.37421557 ...  0.18184404 -0.3489863
  -0.23177223]
 [ 0.89676862  1.01796508  0.96958088 ...  0.95298668 -0.62091216
  -0.65316508]
 [-0.2647643  -0.52432301 -0.71748568 ... -2.44004093  4.0388302
  -1.04743596]]


In [4]:
from DEF_batch import ExtractFeatures

pathIN = "TestTracks.csv"
pathOUT = "TestTracksFeatures.csv"
ExtractFeatures(pathIN,pathOUT)

🎧 Inizio elaborazione delle canzoni
TestTracks.csv TestTracksFeatures.csv Tracks/
OK
🎧 Canzoni totali: 22
✅ Già elaborate: 21
🕐 Da elaborare: 1

🎵 Elaborazione: Pastello Bianco Pinguini Tattici Nucleari (Indie)


Unexpected renderer encountered.
Renderer name: dict_keys(['lockupViewModel'])
Search term: Pastello Bianco Pinguini Tattici Nucleari
Please open an issue at https://github.com/pytube/pytube/issues and provide this log output.
Unexpected renderer encountered.
Renderer name: dict_keys(['reelShelfRenderer'])
Search term: Pastello Bianco Pinguini Tattici Nucleari
Please open an issue at https://github.com/pytube/pytube/issues and provide this log output.
Unexpected renderer encountered.
Renderer name: dict_keys(['reelShelfRenderer'])
Search term: Pastello Bianco Pinguini Tattici Nucleari
Please open an issue at https://github.com/pytube/pytube/issues and provide this log output.


🔍 YouTube URL trovato: https://youtube.com/watch?v=to8uZT8j8UI
⏱️ Durata video: 3 min 58 sec
[youtube] Extracting URL: https://youtube.com/watch?v=to8uZT8j8UI
[youtube] to8uZT8j8UI: Downloading webpage
[youtube] to8uZT8j8UI: Downloading tv client config
[youtube] to8uZT8j8UI: Downloading player 9a279502-main
[youtube] to8uZT8j8UI: Downloading tv player API JSON
[youtube] to8uZT8j8UI: Downloading ios player API JSON
[youtube] to8uZT8j8UI: Downloading m3u8 information
[info] to8uZT8j8UI: Downloading 1 format(s): 251
[download] Destination: Tracks\tmp
[download] 100% of    3.77MiB in 00:00:01 at 3.18MiB/s     
[ExtractAudio] Destination: Tracks\tmp.mp3
Deleting original file Tracks\tmp (pass -k to keep)
✅ Estrazione dei 90 secondi centrali completata.
✅ Download completato con successo.



🎧 Caricamento audio da: Tracks//tmp90.mp3 di 90 sec
🎧 Audio caricato

🎧 Compattamento matrice (13, 8438) in 90 secondi
(13, 90) (13, 90)

🎧 Compattamento matrice (12, 8438) in 90 secondi
(12, 90) (12, 9

In [104]:
import torch.nn.functional as F
songInfo = pd.read_csv(pathIN,encoding="utf-16")
test_df = pd.read_csv(pathOUT)
print(f"LEN:{len(test_df)}")
for r in range(len(test_df)):
    print(f"Riga {r}...")
    nuova_riga_df = test_df.iloc[r]  # Prendi la riga 4 (5a riga) del DataFrame
    #print(nuova_riga_df['id_track'])  # Prendi la prima riga del DataFrame
    new_feature_vector = preprocess_row(nuova_riga_df)  # stessa funzione usata prima
    new_feature_vector = scaler.transform([new_feature_vector])  # normalizzazione
    new_tensor = torch.tensor(new_feature_vector, dtype=torch.float32)

    model.eval()

    with torch.no_grad():
        output = model(new_tensor)
        probabilities = F.softmax(output, dim=1).numpy()[0]  # Applica softmax e ottieni l'array 1D

    print(f"Probabilità: {probabilities}")
    top3_indices = np.argsort(probabilities)[-3:][::-1]  # Ordina e prendi i primi 3 (in ordine decrescente)

    # Mappa gli indici ai nomi dei generi
    predicted_genres = label_encoder.inverse_transform(top3_indices)

    # Mostra i risultati con le probabilità
    print(f"🎧 Genere musicale predetto di: '{songInfo.iloc[r]['title']}' ID {nuova_riga_df['id_track']}: {predicted_genres}")    
    for genre, prob in zip(predicted_genres, probabilities[top3_indices]):
    #st.write(f"{genre}: {prob:.2%}")
        print(f"\t{genre}: {prob*100:.2f}%")

LEN:21
Riga 0...

Preprocessing riga 5840...
Probabilità: [1.49266114e-02 3.19794461e-04 4.56345350e-01 1.22109028e-02
 2.48157373e-03 6.43504923e-03 6.60131201e-02 1.37679090e-05
 1.88662380e-03 2.94507574e-02 7.70735508e-03 9.85356164e-05
 4.23640059e-03 1.63044572e-01 2.22442091e-01 2.67727824e-04
 9.96192284e-06 2.99565261e-03 7.99126457e-03 1.03669496e-04
 1.01924082e-03]
🎧 Genere musicale predetto di: 'Up From The Bottom Linkin Park' ID 0: ['Country' 'Punk' 'Pop']
	Country: 45.63%
	Punk: 22.24%
	Pop: 16.30%
Riga 1...

Preprocessing riga 5841...
Probabilità: [7.1173755e-04 1.2481356e-05 2.1751912e-04 3.2958153e-01 3.8003037e-04
 3.3849198e-04 2.8373498e-02 2.2796883e-04 4.5533932e-05 5.4212976e-02
 1.4431088e-04 1.7324169e-03 3.0905632e-08 4.4426155e-01 3.6825935e-05
 1.3762568e-01 9.5220930e-06 1.0605121e-03 6.6104496e-04 6.6740475e-05
 2.9959509e-04]
🎧 Genere musicale predetto di: 'Beautiful People David Guetta' ID 1: ['Pop' 'Disco' 'R&B']
	Pop: 44.43%
	Disco: 32.96%
	R&B: 13.76

In [ ]:
# torch.save(model, "intero_modello.pth")


In [ ]:
torch.save(model.state_dict(), "modello_sicuro.pth")


In [ ]:
model = MusicGenreClassifier()
model.load_state_dict(torch.load("modello_sicuro.pth"))
model.eval()

UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL __main__.MusicGenreClassifier was not an allowed global by default. Please use `torch.serialization.add_safe_globals([MusicGenreClassifier])` or the `torch.serialization.safe_globals([MusicGenreClassifier])` context manager to allowlist this global if you trust this class/function.

Check the documentation of torch.load to learn more about types a

In [ ]:
test_df = pd.read_csv(pathOUT)
print(f"LEN:{len(test_df)}")
for r in range(len(test_df)):
    print(f"Riga {r}...")
    nuova_riga_df = test_df.iloc[r]  # Prendi la riga 4 (5a riga) del DataFrame
    #print(nuova_riga_df['id_track'])  # Prendi la prima riga del DataFrame
    new_feature_vector = preprocess_row(nuova_riga_df)  # stessa funzione usata prima
    new_feature_vector = scaler.transform([new_feature_vector])  # normalizzazione
    new_tensor = torch.tensor(new_feature_vector, dtype=torch.float32)

    model.eval()

    with torch.no_grad():
        output = model(new_tensor)
        prediction = torch.argmax(output, dim=1)

    predicted_genre = label_encoder.inverse_transform(prediction.numpy())[0]
    print(f"🎧 Genere musicale predetto di ID {nuova_riga_df['id_track']}: {predicted_genre}")